In [ ]:
import os

# List the contents of the input directory
os.listdir('/kaggle/input')


In [2]:
import matplotlib.pyplot as plt
import os 
import cv2
import numpy as np
import seaborn as sns
from keras import models,layers
from keras import optimizers
from keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten
import tensorflow as tf

In [ ]:
def assign_labels(dataset_path):

    images = []
    labels = []
    label_mapping = {}
    
    # Iterate over each subdirectory to get class names
    for idx, class_name in enumerate(sorted(os.listdir(dataset_path))):
        class_path = os.path.join(dataset_path, class_name)
        
        # Only process if it's a directory
        if os.path.isdir(class_path):
            label_mapping[class_name] = idx  # Assign a numeric label to each class
            
            # Load images from the current class directory
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                
                # Read and resize the image, then append to the list
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (128, 128))
                    images.append(img)
                    labels.append(idx)  # Use the numeric label assigned earlier

    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels, label_mapping

# Example usage of assign_labels for train, test, and valid sets
train_dataset_path = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder"
test_dataset_path = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Test_Set_Folder"
valid_dataset_path = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Validation_Set_Folder"

# Assign labels for each set
train_images, train_labels, label_mapping = assign_labels(train_dataset_path)
test_images, test_labels, _ = assign_labels(test_dataset_path)
valid_images, valid_labels, _ = assign_labels(valid_dataset_path)

print("Data loaded successfully.")
print(f"Training samples: {len(train_images)}, Testing samples: {len(test_images)}, Validation samples: {len(valid_images)}")


In [ ]:
label_mapping

In [ ]:
def show_sample_images(images, labels, label_mapping, num_samples=5):
    """
    Display a few sample images with labels.
    
    Args:
        images (np.array): Array of images.
        labels (np.array): Array of labels.
        label_mapping (dict): Dictionary mapping numeric labels to class names.
        num_samples (int): Number of images to display.
    """
    # Reverse the label_mapping to get class names from numeric labels
    class_names = {v: k for k, v in label_mapping.items()}
    
    plt.figure(figsize=(10, 5))
    for i in range(num_samples):
        idx = np.random.randint(0, len(images))
        img = images[idx]
        label = labels[idx]
        label_name = class_names[label]
        
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(label_name)
        plt.axis('off')
    
    plt.show()

# Example usage to display sample images from the train set
show_sample_images(train_images, train_labels, label_mapping)


In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert labels for train, test, and validation sets to categorical format
train_labels_categorical = to_categorical(train_labels, num_classes=len(label_mapping))
test_labels_categorical = to_categorical(test_labels, num_classes=len(label_mapping))
valid_labels_categorical = to_categorical(valid_labels, num_classes=len(label_mapping))

# Verify the shape of the categorical labels
print("Train labels (categorical):", train_labels_categorical.shape)
print("Test labels (categorical):", test_labels_categorical.shape)
print("Validation labels (categorical):", valid_labels_categorical.shape)


In [ ]:
# Print some samples of the converted categorical labels
print("First 5 train labels (categorical):\n", train_labels_categorical[:5])



In [ ]:
# Normalize images to the range [0, 1] by dividing by 255.0
x_train = train_images.astype('float32') / 255.0
x_test = test_images.astype('float32') / 255.0
x_valid = valid_images.astype('float32') / 255.0

# Print the shape to confirm that data is properly prepared
print("Training data shape:", x_train.shape)
print("Testing data shape:", x_test.shape)
print("Validation data shape:", x_valid.shape)


In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dense(30,activation='softmax'))
model.summary()


In [104]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
from time import time

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Record start time
tic = time()

# Train the model
model.fit(
    x_train, train_labels_categorical,
    epochs=15,
    batch_size=100,
    validation_data=(x_valid, valid_labels_categorical),  # Directly pass the validation set
    callbacks=[early_stop]
)

# Print the training time
toc = time()
print("Training time:", toc - tic, "seconds")


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, test_labels_categorical)
print('Test accuracy:', accuracy)

In [ ]:
pred = model.predict(x_test)


In [ ]:
num_images_to_display = 20
num_columns = 4
num_rows = (num_images_to_display + num_columns - 1) // num_columns

fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 10))
for i, ax in enumerate(axes.flat):
    if i < num_images_to_display:
        ax.imshow(x_test[i])
        actual_label = labels[np.argmax(y_test[i])]
        predicted_label = labels[np.argmax(pred[i])]
        ax.set_title(f"Actual: {actual_label}, Predicted: {predicted_label}")
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()
